In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from openmoltools.forcefield_generators import gaffTemplateGenerator
import os

## Finding the structures that are useable
Looking at the latest fixed structures and selecting which ones to minimize with `openmm`. Structures that are inappropriate for MCCE (e.g. contain functionally important cofactors) or are omitted for now, as MCCE can only handle one small molecule at a time.

Some notes about the selection
* Omiting structures with cobimetinib are omitted due to the presence of ATP and an ion in binding site
* Omiting 3PYY as it has an allosteric activator bound.

Manual edits. Files `XXXX-fixed.pdb` were manually edited to make `XXXX-maunal.pdb`.
* Removed duplicate ligands from 4G5J
* Removed chain B from 4XV2.
* Removed extra copied of the inhibitor ligand from 3ZOS
* Renamed duplicate hydrogen atom names (H11 and H12 --> H111 and H112) of P06 in 3CJG
* Renamed duplicate hydrogen name (H11 --> H111) of LEV in 3WZD
* Renamed duplicate hydrogen name (H11 --> H111) of FMM in 1XKK

In [2]:
from glob import glob

# Structures with important cofactors bound. 
omit_pdbs = ['2EUF', '4AN2', '4LMN', '3PYY']

# Structures that need manual refinement
manual_edits = ['4G5J', '4XV2', '3ZOS', '3CJG', '3WZD', '1XKK']   # XXXX-manual.pdb

# For now, neglecting systems that need manual tweaking
omit_pdbs += manual_edits

# Pre-assigment
complete_pdbs = []     # The X-ray structures that are complete, with no modelled in loops
missing_pdbs = []      # The X-ray structures that had missing loops longer than 20 residues, and were not modelled
refine_pdbs = []       # All the structures that will be further refined with OpenMM
refine_pdbs_location = [] # The relative location of the protein-ligand files that will be minimized

for folder in glob('../pdbs/*'):
    for subfolder in glob(folder + '/fixed/*'):
        filename = subfolder.split('/')[-1]
        if filename.split('.')[-1] == 'pdb':
            pdb_code = filename.split('.')[0].split('-')[0]
            #if pdb_code in manual_edits:
            #    decorator = 'manual-'
            #else:
            #    decorator = ''
            if pdb_code not in omit_pdbs:
                logfile = folder + '/fixed/' +  pdb_code + '-fixed' + '/' + pdb_code.lower() + '-missing-loops.log'
                logfile = open(logfile, 'r').read()
                finder = logfile.find('Found loops without PDB coordinates')
                if finder == -1:
                    # If no loops have been modeled, the structure was complete from the start!
                    complete_pdbs.append(pdb_code)
                finder = logfile.find('is too long, skipping')
                if finder != -1:
                    # Disregard structurs whose missing loops were too long to model in
                    missing_pdbs.append(pdb_code)
                else:
                    refine_pdbs.append(pdb_code)
                    refine_pdbs_location.append(folder + '/fixed/' + filename)

# Now removing the duplicate structures that have arison due to the manaully edited PDB structures.
index = 0
for filename in refine_pdbs_location:
    for pdb in manual_edits:
        if  filename.split('/')[-1] == pdb + '-fixed.pdb':
            refine_pdbs.pop(index)
            refine_pdbs_location.pop(index)
    index += 1                    

## Minimizing with OpenMM
And in the process throwing out the structures that can't be minimized.

In [3]:

def discard_organic(model, verbose=True):
    """
    Return an OpenMM modeller object that doesn't contain small organic molecules from the mother liquor
    
    Parameter
    ---------
    model: simtk.openmm.app.modeller.Modeller
        The modeller object with the PDB file of interest
        
    Returns
    -------
    model: simtk.openmm.app.modeller.Modeller
        The same object as the input with certain residues discarded
    """
    unwanted = ['DTT', 'EDO', 'GOL', 'SO4', 'PO4', 'DMS']
    for junk in unwanted:
        atms = [atm for atm in model.topology.atoms() if atm.residue.name == junk]
        if len(atms) > 0:
            if verbose == True: print('Deleting {0} from topology'.format(junk))
            model.delete(atms)
    return model

def get_upper_location(pdb_location, back=2):
    """
    Get the path of the directory that's up from the path specified from pdb_location. 
    """
    path = pdb_location.split('/')[0:-back]
    upper_path = ''
    for location in path:
        upper_path += location + '/'
    return upper_path    

def openmm_clean(pdb_filename, pdbname, out_folder='minimized', solvate=False):
    """
    Minimize a supplied system with openmm. Can handle small molecules as long as CONECT 
    records are supplied.
    """
    # Initialize forcefield with small molecule capabilities
    forcefield = ForceField('gaff.xml','tip3p.xml','amber99sbildn.xml')
    forcefield.registerTemplateGenerator(gaffTemplateGenerator)
    
    # Use modeller to remove unwanted residues
    pdb = PDBFile(pdb_filename)
    model = Modeller(pdb.topology, pdb.positions)
    
    # Remove unwanted molecules
    model = discard_organic(model, verbose=False)
    
    # Add waters in a cubic box
    if solvate == True:
        model.addSolvent(forcefield, padding=1.0*nanometers)
    
    # Create the system with a cheap electrostatic cutoff
    system = forcefield.createSystem(model.topology, nonbondedMethod=CutoffNonPeriodic)
    
    # Minimize system with a placeholder integrator
    integrator = VerletIntegrator(0.001*picoseconds)
    simulation = Simulation(model.topology, system, integrator)
    simulation.context.setPositions(model.positions)
    simulation.minimizeEnergy()
    
    # Print PDB
    positions = simulation.context.getState(getPositions=True).getPositions()
    out_directory = get_upper_location(pdb_filename)
    try:
        os.mkdir(out_directory + out_folder)
    except OSError:
        pass
    PDBFile.writeFile(simulation.topology, positions, open(out_directory + out_folder + pdbname + '-minimized.pdb', 'w'))

In [1]:
import logger

ImportError: No module named logger

In [4]:
import warnings
warnings.filterwarnings('error')

import logging
logging.basicConfig(filename='explicit_solvent_minimization.log',level=logging.DEBUG)


In [5]:
# Pre-assigment
broken_pdbs = []
minimized_pdbs = []
minimized_files = []

# Running through refined structures and seeing if they work
for filename, pdb in zip(refine_pdbs_location, refine_pdbs):
    #print 'Running with {0}'.format(pdb)
    try:
        openmm_clean(filename, pdb)
        minimized_pdbs.append(pdb)
        minimized_files.append(filename)
    except Exception as detail:
        #print detail
        broken_pdbs.append(filename)

## Results

This files incurred further errors with `openmm`:

In [6]:
for filename in broken_pdbs:
    print filename 

../pdbs/Dabrafenib-BRAF/fixed/4XV2-fixed.pdb
../pdbs/Dabrafenib-BRAF/fixed/5CSW-fixed.pdb
../pdbs/Dabrafenib-BRAF/fixed/5HIE-fixed.pdb
../pdbs/Lapatinib-EGFR/fixed/1XKK-fixed.pdb
../pdbs/Lenvatinib-VEGFR1/fixed/3WZD-fixed.pdb
../pdbs/Pazopanib-VEGFR1/fixed/3CJG-fixed.pdb


The following files are passed the current tests, but will need further inspection.

In [7]:
for filename in minimized_files:
    print filename

../pdbs/Alectinib-ALK/fixed/3AOX-fixed.pdb
../pdbs/Axitinib-VEGFR1/fixed/4AG8-fixed.pdb
../pdbs/Axitinib-VEGFR1/fixed/4AGC-fixed.pdb
../pdbs/Bosutinib-BCR-ABL/fixed/3UE4-fixed.pdb
../pdbs/Ceritinib-ALK/fixed/4MKC-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/2XP2-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/2YFX-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/4ANQ-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/4ANS-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/5AAA-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/5AAB-fixed.pdb
../pdbs/Crizotinib-ALK/fixed/5AAC-fixed.pdb
../pdbs/Crizotinib-MET/fixed/2WGJ-fixed.pdb
../pdbs/Dasatinib-BCR-ABL/fixed/2GQG-fixed.pdb
../pdbs/Dasatinib-BCR-ABL/fixed/4XEY-fixed.pdb
../pdbs/Erlotinib-EGFR/fixed/1M17-fixed.pdb
../pdbs/Erlotinib-EGFR/fixed/4HJO-fixed.pdb
../pdbs/Gefitinib-EGFR/fixed/2ITO-fixed.pdb
../pdbs/Gefitinib-EGFR/fixed/2ITY-fixed.pdb
../pdbs/Gefitinib-EGFR/fixed/2ITZ-fixed.pdb
../pdbs/Gefitinib-EGFR/fixed/3UG2-fixed.pdb
../pdbs/Gefitinib-EGFR/fixed/4I22-fixed.pdb
../pdbs/Imatinib-BCR-AB